In [69]:
# 导入分析需要的包
import pandas as pd
import numpy as np
from pyecharts import options as opts
import os
import pyecharts
from pyecharts.charts import *

In [81]:
# 引入需要的数据
path = "C:/Users/13128/Desktop/machine learning/education"
wb_list = []
for root,dirs,files in os.walk(path):
    wb_list.append(files)
wb_list = wb_list[0]
print(wb_list)
data = []
for i in range(len(wb_list)):
    path1 = path + "/" + str(wb_list[i])
    data.append(pd.read_csv(path1))
teacher = data[0]
student_info = data[1]
kaoqin = data[2]
kaoqintype = data[3]
chengji = data[4]
exam_type = data[5]
consumption = data[6]

['1_teacher.csv', '2_student_info.csv', '3_kaoqin.csv', '4_kaoqintype.csv', '5_chengji.csv', '6_exam_type.csv', '7_consumption.csv']


In [82]:
# 将考试时间由字符串转换为时间格式
chengji["exam_sdate"] = pd.to_datetime(chengji["exam_sdate"])

In [83]:
# 添加一列填充为考试年份
chengji["year"] = chengji["exam_sdate"].dt.year

In [84]:
# 去掉分析不需要的列
chengji.drop(['mes_Z_Score', 'mes_T_Score', 'mes_dengdi'],axis = 1,inplace = True)

In [85]:
# 去重
chengji.drop_duplicates(inplace = True)

In [86]:
# 删除错误值和空值
chengji.drop(index = chengji[chengji["mes_sub_id"].isnull()].index,inplace = True)

In [87]:
# 将缺考和作弊的考生成绩填充为0，将免考的学生成绩填充为该场考试的最高分
chengji["mes_Score"].replace(-1,0,inplace = True)
chengji["mes_Score"].replace(-2,0,inplace = True)
zuigaofen = chengji.groupby(by = ["mes_TestID","mes_sub_id"])["mes_Score"].max().reset_index(name = "最高分")
for i in chengji["mes_TestID"].drop_duplicates().values:
    chengji.loc[chengji["mes_TestID"] == i,"mes_Score"].replace(-3,zuigaofen.loc[zuigaofen["mes_TestID"]  == i,"最高分"].values[0],inplace = True)

In [88]:
# 按考试年份和学生ID对考试分数汇总
total_score = chengji.groupby(by = ["year","mes_StudentID"])["mes_Score"].sum().reset_index(name = "总分")

In [89]:
# 将考勤表中的考勤时间从字符串转换为时间格式，并添加一列考勤年份
kaoqin["DataDateTime"] = pd.to_datetime(kaoqin["DataDateTime"] )
kaoqin["year"]  = kaoqin["DataDateTime"].dt.year

In [90]:
# 按考勤年份和学生ID对缺勤次数进行汇总
queqin  = kaoqin.groupby(by = ["year","bf_studentID"])["controler_name"].count().reset_index(name = "queqin")

In [91]:
# 连接前两次汇总表并再次按照缺勤次数对考试成绩总分进行汇总
fenxi_data = pd.merge(total_score,queqin,left_on = ["year","mes_StudentID"],right_on = ["year","bf_studentID"],how = "inner")
fenxi_datag = fenxi_data.groupby(by = ["queqin"])["总分"].sum().reset_index(name = "总分")

In [92]:
# 对缺勤次数和考试成绩进行可视化，可以明显看出随着缺勤次数的增多，考试成绩会随之降低，因此学校应大力抓好考勤工作。
from pyecharts import *
x_data = [int(i) for i in fenxi_datag["queqin"].values]
y_data = [int(i) for i in fenxi_datag["总分"].values]
line = (
       Line()
        .add_xaxis(x_data)
        .add_yaxis("",y_data,label_opts = opts.LabelOpts(is_show = False))
        .set_global_opts(xaxis_opts = opts.AxisOpts())
)
line.render_notebook()